In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
from sklearn.model_selection import train_test_split


In [7]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 243909337249068849
]


In [8]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


[]

In [9]:
def create_train_validation(file_path):
    data = pd.read_csv(file_path)
    train_set, validation_set = train_test_split(data, test_size=0.1)
    train_set.reset_index(inplace = True)
    validation_set.reset_index(inplace = True)
    return train_set, validation_set


train_df, validation_df = create_train_validation('food-recognition-challenge/train_labels.csv')
train_df['label'] = train_df['label'].astype(str)
validation_df['label'] = validation_df['label'].astype(str)
# df = pd.read_csv('food-recognition-challenge/train_labels.csv')


,index,img_name,label
0,25135,train_25136.jpg,5
1,10532,train_10533.jpg,18
2,11113,train_11114.jpg,65
3,6399,train_6400.jpg,45
4,5635,train_5636.jpg,15
5,24776,train_24777.jpg,47
6,3042,train_3043.jpg,53
7,12957,train_12958.jpg,8
8,13067,train_13068.jpg,2
9,9879,train_9880.jpg,48


In [2]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory='food-recognition-challenge/train_set/train_set/',
        x_col="img_name",
        y_col="label",
        target_size=(256, 256),
        batch_size=32,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_dataframe(
        dataframe=validation_df,
        directory='food-recognition-challenge/train_set/train_set/',
        x_col="img_name",
        y_col="label",
        target_size=(256, 256),
        batch_size=32,
        class_mode='categorical')

/Users/olafkroon/anaconda3/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:355: UserWarning: This ImageDataGenerator specifies `samplewise_std_normalization`, which overrides setting of `samplewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


Found 27550 validated image filenames belonging to 80 classes.
Found 3062 validated image filenames belonging to 80 classes.


In [ ]:
import tensorflow as tf

def outer_product(x):
    #Einstein Notation  [batch,1,1,depth] x [batch,1,1,depth] -> [batch,depth,depth]
    phi_I = tf.einsum('ijkm,ijkn->imn',x[0],x[1])
    
    # Reshape from [batch_size,depth,depth] to [batch_size, depth*depth]
    phi_I = tf.reshape(phi_I,[-1,x[0].shape[3]*x[1].shape[3]])
    
    # Divide by feature map size [sizexsize]
    size1 = int(x[1].shape[1])
    size2 = int(x[1].shape[2])
    phi_I = tf.divide(phi_I, size1*size2)
    
    # Take signed square root of phi_I
    y_ssqrt = tf.multiply(tf.sign(phi_I),tf.sqrt(tf.abs(phi_I)+1e-12))
    
    # Apply l2 normalization
    z_l2 = tf.nn.l2_normalize(y_ssqrt)
    return z_l2

In [ ]:

from keras.models import Sequential
from keras.layers import Dense, Dropout, Lambda
from keras.models import Model
from keras.applications.vgg16 import VGG16

# load the model , input_shape= (3,256,256)
model1 = VGG16(weights='imagenet', include_top=False, input_shape = (256, 256, 3))
# Freeze the layers except the last 4 layers
for layer in model1.layers:
    layer.trainable = False
 

for layer in model1.layers[:-1]:
    print(layer, layer.trainable)

conv=model1.get_layer('block4_conv3') # block4_conv3
d1=Dropout(0.5)(conv.output)
d2=Dropout(0.5)(conv.output)

x = Lambda(outer_product, name='outer_product')([d1,d2])
predictions=Dense(80, activation='softmax', name='predictions')(x)

model = Model(inputs=model1.input, outputs=predictions)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=2000,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=800)

In [ ]:
model.save('b-cnn.h5')  # creates a HDF5 file 'my_model.h5'
